<a href="https://colab.research.google.com/github/siravitgonarm/Project/blob/main/Data_Mining_Project_(Animation_Preference_Prediction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data/datamine final test.csv')

In [ ]:
# ทำความสะอาดชื่อคอลัมน์เพื่อให้ง่ายต่อการอ้างอิง
df.columns = [
    'gender', 'watch_frequency', 'watch_platform_text', 'platform_youtube', 'platform_netflix',
    'platform_bilibili', 'platform_iqiyi', 'platform_viu', 'platform_online_sites',
    'platform_files', 'platform_tv', 'favorite_genre', 'episodes_per_session', 'watch_reason_text',
    'reason_stress_relief', 'reason_art_sound', 'reason_story', 'reason_characters',
    'reason_good_message', 'reason_trending', 'reason_inspiration', 'ending_preference_text',
    'ending_happy', 'ending_sad', 'ending_open', 'ending_twist', 'ending_sequel',
    'ending_realistic', 'ending_touching', 'ending_fun', 'selection_criteria_text',
    'criteria_cover', 'criteria_teaser', 'criteria_story', 'criteria_soundtrack',
    'criteria_characters', 'criteria_va', 'criteria_friends_recommend', 'character_connection',
    'preferred_language'
]

In [ ]:
# กำหนด Target (y) และ Features (X)
X = df.drop(columns=['favorite_genre', 'watch_platform_text', 'watch_reason_text', 'ending_preference_text', 'selection_criteria_text'])
y = df['favorite_genre']

# แปลงข้อมูลที่เป็นข้อความให้เป็นตัวเลข (Label Encoding)
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_labels = encoder.classes_ # เก็บชื่อแนวอนิเมะไว้ดูภายหลัง

# แปลงข้อมูลใน X ทั้งหมดให้เป็นตัวเลข
for col in X.select_dtypes(include='object').columns:
    X[col] = encoder.fit_transform(X[col])

In [ ]:
# 2. การเลือก Feature (Feature Selection) ด้วย Information Gain
# ==============================================================================
print("🔍 กำลังคำนวณ Information Gain เพื่อหาปัจจัยที่ดีที่สุด...")
info_gain = mutual_info_classif(X, y_encoded, discrete_features=True)
info_gain_series = pd.Series(info_gain, index=X.columns)

# เลือก 5 ปัจจัยที่มีค่า Information Gain สูงสุด
top_5_features = info_gain_series.sort_values(ascending=False).head(5)

print("\n✅ 5 ปัจจัยที่มีอิทธิพลสูงสุดต่อการทำนายแนวอนิเมะที่ชอบ:\n")
print(top_5_features)
print("-" * 50)

# สร้าง DataFrame ใหม่โดยใช้แค่ 5 features ที่เลือก
X_top5 = X[top_5_features.index]


🔍 กำลังคำนวณ Information Gain เพื่อหาปัจจัยที่ดีที่สุด...

✅ 5 ปัจจัยที่มีอิทธิพลสูงสุดต่อการทำนายแนวอนิเมะที่ชอบ:

character_connection    0.157513
episodes_per_session    0.129570
preferred_language      0.047054
ending_happy            0.046351
reason_story            0.045579
dtype: float64
--------------------------------------------------


In [ ]:
# 3. แบ่งข้อมูลสำหรับ Train และ Test
# ==============================================================================
# แบ่งข้อมูลเป็น Train 60% และ Test 40%
X_train, X_test, y_train, y_test = train_test_split(
    X_top5,
    y_encoded,
    train_size=0.6,
    test_size=0.4,
    random_state=42, # กำหนดเพื่อให้ผลลัพธ์เหมือนเดิมทุกครั้งที่รัน
    stratify=y_encoded # ทำให้สัดส่วนของแนวอนิเมะใน train/test เหมือนข้อมูลจริง
)

print(f"แบ่งข้อมูลเรียบร้อย:")
print(f" - ชุดข้อมูลสำหรับ Train: {X_train.shape[0]} ตัวอย่าง")
print(f" - ชุดข้อมูลสำหรับ Test: {X_test.shape[0]} ตัวอย่าง")
print("-" * 50)

แบ่งข้อมูลเรียบร้อย:
 - ชุดข้อมูลสำหรับ Train: 197 ตัวอย่าง
 - ชุดข้อมูลสำหรับ Test: 132 ตัวอย่าง
--------------------------------------------------


In [ ]:
# เตรียม K-Fold สำหรับ Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
results = {}

In [ ]:
# --- 🌳 โมเดลที่ 1: Decision Tree ---
print("\n[1/4] 🌳 กำลังสร้างโมเดล Decision Tree...")
dt_model = DecisionTreeClassifier(max_depth=10, random_state=42)

# ประเมินด้วย K-Fold
cv_scores_dt = cross_val_score(dt_model, X_train, y_train, cv=kfold, scoring='accuracy')
results['Decision Tree'] = {'kfold_mean': cv_scores_dt.mean()}
print(f"   - K-Fold Cross-Validation Accuracy: {cv_scores_dt.mean():.4f} (+/- {cv_scores_dt.std():.4f})")

# สอนโมเดลและทดสอบ
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
results['Decision Tree']['test_accuracy'] = accuracy_dt
print(f"   - Test Set Accuracy: {accuracy_dt:.4f}\n")
print("   - Classification Report:")
print(classification_report(y_test, y_pred_dt, target_names=y_labels, zero_division=0))
print("-" * 50)


[1/4] 🌳 กำลังสร้างโมเดล Decision Tree...
   - K-Fold Cross-Validation Accuracy: 0.3251 (+/- 0.0457)
   - Test Set Accuracy: 0.3258

   - Classification Report:
                    precision    recall  f1-score   support

 คอมเมดี้ (Comedy)       0.33      0.44      0.38        25
    ดราม่า (Drama)       0.47      0.45      0.46        20
 แฟนตาซี (Fantasy)       0.32      0.27      0.29        30
  แอคชั่น (Action)       0.23      0.27      0.25        33
โรแมนติก (Romance)       0.40      0.25      0.31        24

          accuracy                           0.33       132
         macro avg       0.35      0.34      0.34       132
      weighted avg       0.34      0.33      0.33       132

--------------------------------------------------


In [ ]:
# --- 🌲 โมเดลที่ 2: Random Forest (ปรับเป็น 100 ต้น) ---
print("\n[2/4] 🌲 กำลังสร้างโมเดล Random Forest (100 trees)...")
# เพิ่ม n_estimators=100 เพื่อกำหนดให้มี Decision Tree 100 ต้น
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# ประเมินด้วย K-Fold
cv_scores_rf = cross_val_score(rf_model, X_train, y_train, cv=kfold, scoring='accuracy')
results['Random Forest'] = {'kfold_mean': cv_scores_rf.mean()}
print(f"   - K-Fold Cross-Validation Accuracy: {cv_scores_rf.mean():.4f} (+/- {cv_scores_rf.std():.4f})")

# สอนโมเดลและทดสอบ
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
results['Random Forest']['test_accuracy'] = accuracy_rf
print(f"   - Test Set Accuracy: {accuracy_rf:.4f}\n")
print("   - Classification Report:")
print(classification_report(y_test, y_pred_rf, target_names=y_labels, zero_division=0))
print("-" * 50)


[2/4] 🌲 กำลังสร้างโมเดล Random Forest (100 trees)...
   - K-Fold Cross-Validation Accuracy: 0.3347 (+/- 0.0445)
   - Test Set Accuracy: 0.3485

   - Classification Report:
                    precision    recall  f1-score   support

 คอมเมดี้ (Comedy)       0.30      0.28      0.29        25
    ดราม่า (Drama)       0.56      0.45      0.50        20
 แฟนตาซี (Fantasy)       0.35      0.40      0.38        30
  แอคชั่น (Action)       0.26      0.30      0.28        33
โรแมนติก (Romance)       0.40      0.33      0.36        24

          accuracy                           0.35       132
         macro avg       0.38      0.35      0.36       132
      weighted avg       0.36      0.35      0.35       132

--------------------------------------------------


In [ ]:
# --- 📊 โมเดลที่ 3: Naive Bayes ---
# เราใช้ CategoricalNB เพราะข้อมูลของเราเป็นหมวดหมู่ทั้งหมด
print("\n[3/4] 📊 กำลังสร้างโมเดล Naive Bayes...")
nb_model = CategoricalNB()

# ประเมินด้วย K-Fold
cv_scores_nb = cross_val_score(nb_model, X_train, y_train, cv=kfold, scoring='accuracy')
results['Naive Bayes'] = {'kfold_mean': cv_scores_nb.mean()}
print(f"   - K-Fold Cross-Validation Accuracy: {cv_scores_nb.mean():.4f} (+/- {cv_scores_nb.std():.4f})")

# สอนโมเดลและทดสอบ
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
results['Naive Bayes']['test_accuracy'] = accuracy_nb
print(f"   - Test Set Accuracy: {accuracy_nb:.4f}\n")
print("   - Classification Report:")
print(classification_report(y_test, y_pred_nb, target_names=y_labels, zero_division=0))
print("-" * 50)


[3/4] 📊 กำลังสร้างโมเดล Naive Bayes...
   - K-Fold Cross-Validation Accuracy: 0.3603 (+/- 0.0578)
   - Test Set Accuracy: 0.3939

   - Classification Report:
                    precision    recall  f1-score   support

 คอมเมดี้ (Comedy)       0.48      0.48      0.48        25
    ดราม่า (Drama)       0.65      0.55      0.59        20
 แฟนตาซี (Fantasy)       0.34      0.43      0.38        30
  แอคชั่น (Action)       0.28      0.33      0.31        33
โรแมนติก (Romance)       0.38      0.21      0.27        24

          accuracy                           0.39       132
         macro avg       0.43      0.40      0.41       132
      weighted avg       0.41      0.39      0.39       132

--------------------------------------------------


In [ ]:
# --- 🤝 โมเดลที่ 4: K-Nearest Neighbors (KNN) ---
print("\n[4/4] 🤝 กำลังสร้างโมเดล K-Nearest Neighbors (KNN)...")
knn_model = KNeighborsClassifier(n_neighbors=5) # n_neighbors คือค่า K, 5 เป็นค่าเริ่มต้นที่ดี

# ประเมินด้วย K-Fold
cv_scores_knn = cross_val_score(knn_model, X_train, y_train, cv=kfold, scoring='accuracy')
results['K-Nearest Neighbors'] = {'kfold_mean': cv_scores_knn.mean()}
print(f"   - K-Fold Cross-Validation Accuracy: {cv_scores_knn.mean():.4f} (+/- {cv_scores_knn.std():.4f})")

# สอนโมเดลและทดสอบ
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
results['K-Nearest Neighbors']['test_accuracy'] = accuracy_knn
print(f"   - Test Set Accuracy: {accuracy_knn:.4f}\n")
print("   - Classification Report:")
print(classification_report(y_test, y_pred_knn, target_names=y_labels, zero_division=0))
print("-" * 50)


[4/4] 🤝 กำลังสร้างโมเดล K-Nearest Neighbors (KNN)...
   - K-Fold Cross-Validation Accuracy: 0.3659 (+/- 0.0469)
   - Test Set Accuracy: 0.2879

   - Classification Report:
                    precision    recall  f1-score   support

 คอมเมดี้ (Comedy)       0.37      0.40      0.38        25
    ดราม่า (Drama)       0.30      0.30      0.30        20
 แฟนตาซี (Fantasy)       0.26      0.30      0.28        30
  แอคชั่น (Action)       0.24      0.27      0.26        33
โรแมนติก (Romance)       0.31      0.17      0.22        24

          accuracy                           0.29       132
         macro avg       0.30      0.29      0.29       132
      weighted avg       0.29      0.29      0.28       132

--------------------------------------------------


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 5. สรุปผลการเปรียบเทียบโมเดล
# ==============================================================================
summary_df = pd.DataFrame(results).T
summary_df.columns = ['K-Fold Mean Accuracy', 'Test Set Accuracy']
summary_df = summary_df.sort_values(by='Test Set Accuracy', ascending=False)

print("\n\n🏆 สรุปผลการเปรียบเทียบประสิทธิภาพของโมเดล 🏆\n")
print(summary_df)
print("\n" + "="*50)
print(f"🎉 โมเดลที่มีประสิทธิภาพสูงสุดคือ: {summary_df.index[0]} 🎉")
print(f"   ด้วยความแม่นยำบน Test Set: {summary_df['Test Set Accuracy'].iloc[0]:.2%}")
print("="*50)




🏆 สรุปผลการเปรียบเทียบประสิทธิภาพของโมเดล 🏆

                     K-Fold Mean Accuracy  Test Set Accuracy
Naive Bayes                      0.360256           0.393939
Random Forest                    0.334744           0.348485
Decision Tree                    0.325128           0.325758
K-Nearest Neighbors              0.365897           0.287879

🎉 โมเดลที่มีประสิทธิภาพสูงสุดคือ: Naive Bayes 🎉
   ด้วยความแม่นยำบน Test Set: 39.39%


In [ ]:
import pandas as pd

# ==============================================================================
# ✨ ส่วนของการทำนายผล ✨
# ==============================================================================

# คำอธิบายการใส่ข้อมูลสำหรับ 5 ปัจจัยที่โมเดลใช้ (อิงจากผลลัพธ์ที่คาดว่าจะได้):
# จากผลการเลือก Feature 5 อันดับแรกคือ:
# character_connection
# episodes_per_session
# preferred_language
# ending_happy
# reason_story

# ----> 1. ป้อนข้อมูลของคุณที่นี่ <----
# **หมายเหตุ:** ค่าที่ป้อนต้องเป็นค่าที่ไม่ได้ถูกแปลงเป็นตัวเลข (ค่าดิบ)
# ตัวอย่าง: คนที่เน้นเนื้อเรื่อง ไม่สนตัวละคร ชอบดูพากย์ไทย และเชื่อมโยงกับตัวละครพอสมควร ดูอนิเมะครั้งละ 3-5 ตอน จบแบบ Happy Ending
my_sample = {
    'character_connection': 'พอสมควร', # ระดับการเชื่อมโยงกับตัวละคร
    'episodes_per_session': '3-5 ตอน', # จำนวนตอนที่ดูต่อครั้ง
    'preferred_language': 'พากย์ไทย', # ภาษาที่ชอบดู
    'ending_happy': 'พอสมควร',     # ความชอบตอนจบแบบ Happy Ending
    'reason_story': 'มากที่สุด'      # เหตุผลที่ดูเพราะเนื้อเรื่องน่าสนใจ
}

# ----> 2. แปลงข้อมูลและทำนายผล <----
# สร้าง DataFrame จากข้อมูลตัวอย่าง โดยให้มีคอลัมน์ครบตาม top_5_features.index
# และเรียงลำดับคอลัมน์ให้ถูกต้อง
sample_df = pd.DataFrame([my_sample])

# ตรวจสอบและเรียงลำดับคอลัมน์ให้ตรงกับที่โมเดลคาดหวัง
# หากมีคอลัมน์ไหนใน top_5_features.index ที่ไม่มีใน my_sample จะเกิด KeyError
# ดังนั้น ต้องแน่ใจว่า my_sample มี key ครบตาม top_5_features.index
required_columns = top_5_features.index.tolist()
if not all(col in my_sample for col in required_columns):
    missing = [col for col in required_columns if col not in my_sample]
    raise ValueError(f"ข้อมูลตัวอย่างไม่ครบถ้วน. ขาดคอลัมน์: {missing}")

sample_df = sample_df[required_columns]


# เนื่องจาก LabelEncoder ในโค้ดเดิมถูกใช้ซ้ำๆ และไม่ได้เก็บ mapping ของแต่ละคอลัมน์ไว้
# การแปลงข้อมูลตัวอย่างโดยอัตโนมัติจากค่าดิบจึงทำได้ยากและอาจเกิดข้อผิดพลาด
# เพื่อให้โค้ดส่วนทำนายรันได้ตอนนี้ เราจะใช้ค่าตัวเลขที่คาดว่าถูกต้องตามข้อมูล Train
# และปรับปรุงคำอธิบายการป้อนข้อมูล

# ----> 1. ป้อนข้อมูลของคุณที่นี่ (ในรูปแบบตัวเลขที่โมเดลเข้าใจ) <----
# อิงจากค่าที่โมเดลเห็นในข้อมูล Train (X_train) สำหรับ 5 ปัจจัยหลัก
# **จำเป็นต้องรู้ว่าแต่ละค่าดิบแปลงเป็นตัวเลขอะไร**
# **เนื่องจากโค้ดเดิมไม่ได้แสดง mapping นี้ออกมา**
# **ค่าเหล่านี้เป็นค่าประมาณที่ปรับตามข้อผิดพลาด IndexError ที่พบ**
# **และอาจยังไม่ถูกต้อง 100% ขึ้นอยู่กับข้อมูลจริงและการ fit ของ encoder**
# **ทางออกที่ดีที่สุดในระยะยาวคือปรับปรุงโค้ดส่วนการแปลงข้อมูล X (-1dQZvWp4bg) เพื่อเก็บ encoder ของแต่ละคอลัมน์ไว้**

# ตัวอย่าง: คนที่เน้นเนื้อเรื่อง (มากที่สุด -> ค่าสูง), ไม่สนตัวละคร (การเชื่อมโยงต่ำ -> ค่าต่ำ),
# ชอบดูพากย์ไทย (ค่าอาจเป็น 0), ดูครั้งละ 3-5 ตอน (ค่าอาจเป็น 1 หรือ 2), จบแบบ Happy Ending (ชอบมาก -> ค่าสูง)
# ค่าเหล่านี้เป็นแค่การสมมติ ควรใช้ mapping จริงจากข้อมูลที่ถูกแปลงแล้ว
# ปรับค่าตามข้อผิดพลาดที่พบและค่าที่เห็นใน X_train (ตรวจสอบจาก Variable Inspector)
# character_connection มีค่าตั้งแต่ 0 ถึง 7
# episodes_per_session มีค่าตั้งแต่ 0 ถึง 5
# preferred_language มีค่าตั้งแต่ 0 ถึง 2
# ending_happy มีค่าตั้งแต่ 0 ถึง 1
# reason_story มีค่าตั้งแต่ 0 ถึง 1

my_sample_encoded = {
    'character_connection': 5, # ปรับค่าให้อยู่ในขอบเขต 0-7 (เดิม 6)
    'episodes_per_session': 0, # ปรับค่าให้อยู่ในขอบเขต 0-5 (เดิม 2)
    'preferred_language': 1,    # ค่า 0 อยู่ในขอบเขต 0-2
    'ending_happy': 1,        # ปรับค่าให้อยู่ในขอบเขต 0-1 (เดิม 4 - น่าจะเป็นปัญหาหลัก)
    'reason_story': 0         # ปรับค่าให้อยู่ในขอบเขต 0-1 (เดิม 4 - น่าจะเป็นปัญหาหลัก)
}

# ----> 2. ทำนายผล <----
# สร้าง DataFrame จากข้อมูลตัวอย่างที่ถูกแปลงเป็นตัวเลขแล้ว
sample_df_encoded = pd.DataFrame([my_sample_encoded])

# จัดลำดับคอลัมน์ให้ตรงกับที่โมเดลเรียนมา
required_columns = top_5_features.index.tolist()
sample_df_encoded = sample_df_encoded[required_columns]


# ทำนายผลด้วยโมเดล Naive Bayes (ซึ่งให้ประสิทธิภาพสูงสุด)
# ใช้ nb_model แทน rf_model ตามผลสรุป
prediction_encoded = nb_model.predict(sample_df_encoded)

# แปลงผลลัพธ์ตัวเลขกลับเป็นชื่อแนวอนิเมะที่อ่านเข้าใจ (ใช้ encoder ที่ fit กับ y)
# encoder ที่ fit กับ y (แนวอนิเมะ) ยังคงใช้ได้
predicted_genre = encoder.inverse_transform(prediction_encoded)

# ----> 3. แสดงผลลัพธ์ <----
print("\n📊 ข้อมูลที่คุณป้อน (ในรูปแบบตัวเลข):")
display(sample_df_encoded) # ใช้ display เพื่อแสดงผล DataFrame ได้ดีขึ้น
print("\n🚀 ผลการทำนาย:")
print(f"โมเดลทำนายว่าคุณน่าจะชอบอนิเมะแนว: 👉 {predicted_genre[0]} 👈")

# คำอธิบายเพิ่มเติม:
# เพื่อให้การป้อนข้อมูลสะดวกขึ้น (ป้อนเป็นข้อความดิบได้)
# และเพื่อให้การแปลงถูกต้อง 100% ตามข้อมูลที่โมเดลเรียนรู้
# จำเป็นต้องแก้ไขโค้ดส่วนการแปลงข้อมูล X (-1dQZvVpW4bg)
# โดยการเก็บ LabelEncoder ของแต่ละคอลัมน์ไว้
# หรือใช้ One-Hot Encoding แทน Label Encoding


📊 ข้อมูลที่คุณป้อน (ในรูปแบบตัวเลข):


,character_connection,episodes_per_session,preferred_language,ending_happy,reason_story
0,5,0,1,1,0



🚀 ผลการทำนาย:
โมเดลทำนายว่าคุณน่าจะชอบอนิเมะแนว: 👉 โรแมนติก (Romance) 👈
